In [28]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [29]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 64 # 1st layer num features
num_hidden_2 = 16 # 2nd layer num features (the latent dim)
num_input = 21 
num_classes = 5

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_2, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [30]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [31]:
from numpy import genfromtxt
my_data = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VVc.csv', delimiter=',')

In [32]:
data = my_data[1:,24:]
print(data.shape)

(291, 21)


In [33]:
print(data.max())

0.746019959449768


In [34]:
data = np.array(data)
print(data.shape)

(291, 21)


In [35]:
import pandas
colnames = ['Latitude', 'Longitude', 'CType', 'Remarks', 'Index']
data1 = pandas.read_csv('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/CropClean.csv', names=colnames)

In [36]:
ctype = data1.CType.tolist()
print(ctype)

['CType', 'PD', 'PD', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'SC', 'SC', 'SC', 'JW', 'PD', 'SC', 'BA', 'SC', 'BA', 'PD', 'PD', 'SC', 'PP', 'PD', 'PD', 'PD', 'PD', 'PD', 'BA', 'SC', 'SC', 'BA', 'BA', 'JW', 'JW', 'SC', 'PD', 'BA', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'BA', 'SC', 'PD', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'PD', 'PD', 'SC', 'JW', 'PD', 'PD', 'BA', 'BA', 'SC', 'SC', 'SC', 'BA', 'PP', 'PP', 'BA', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'PD', 'PD', 'SC', 'PD', 'SC', 'PD', 'BA', 'JW', 'PD', 'SC', 'JW', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'SC', 'SC', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'JW', 'PD', 'SC', 'SC', 'JW', 

In [37]:
p = np.array(ctype)
print(p.shape)

(292,)


In [38]:
lbl1 = ctype
label = []
for i in range(1,292):
    if(lbl1[i] == "PD"):
        label += [0]
    if(lbl1[i] == "JW"):
        label += [1]
    if(lbl1[i] == "PP"):
        label += [2]
    if(lbl1[i] == "SC"):
        label += [3]
    if(lbl1[i] == "BA"):
        label += [4]
print(label)

[0, 0, 0, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 1, 0, 3, 4, 3, 4, 0, 0, 3, 2, 0, 0, 0, 0, 0, 4, 3, 3, 4, 4, 1, 1, 3, 0, 4, 1, 3, 3, 3, 3, 3, 0, 0, 4, 3, 0, 0, 3, 3, 3, 3, 3, 1, 0, 0, 3, 1, 0, 0, 4, 4, 3, 3, 3, 4, 2, 2, 4, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 0, 0, 3, 0, 3, 0, 4, 1, 0, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 1, 3, 3, 3, 3, 3, 3, 0, 1, 0, 3, 3, 1, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 4, 4, 3, 0, 4, 0, 3, 3, 3, 3, 0, 3, 0, 3, 2, 3, 0, 3, 1, 4, 3, 0, 0, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 4, 1, 0, 0, 2, 3, 3, 3, 0, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 0, 4, 4, 4, 0, 0, 0, 0, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 3, 1, 3, 4, 1, 3, 3, 3, 3, 4, 0, 0]


In [39]:
label = np.array(label)
print(label.shape)

(291,)


In [40]:
# 80% train 20% test (NDVIclean)
X_train = data[0:232,:]
X_test = data[232:,:]
y_train = label[0:232]
y_test = label[232:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(232, 21)
(59, 21)
(232,)
(59,)


In [41]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(232, 5)
(59, 5)


In [42]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 13

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.31475129
Epoch: 2 cost = 1.13531725
Epoch: 3 cost = 1.06115946
Epoch: 4 cost = 1.02530747
Epoch: 5 cost = 0.99624626
Epoch: 6 cost = 0.96591792
Epoch: 7 cost = 0.93302448
Epoch: 8 cost = 0.89824339
Epoch: 9 cost = 0.86288581
Epoch: 10 cost = 0.82815442
Epoch: 11 cost = 0.79469688
Epoch: 12 cost = 0.76259041
Epoch: 13 cost = 0.73158763
Epoch: 14 cost = 0.70136668
Epoch: 15 cost = 0.67167818
Epoch: 16 cost = 0.64242101
Epoch: 17 cost = 0.61368898
Epoch: 18 cost = 0.58577985
Epoch: 19 cost = 0.55909835
Epoch: 20 cost = 0.53392021
Epoch: 21 cost = 0.51019504
Epoch: 22 cost = 0.48766461
Epoch: 23 cost = 0.46620892
Epoch: 24 cost = 0.44604213
Epoch: 25 cost = 0.42763195
Epoch: 26 cost = 0.41148626
Epoch: 27 cost = 0.39794707
Epoch: 28 cost = 0.38707486
Epoch: 29 cost = 0.37865327
Epoch: 30 cost = 0.37227842
Epoch: 31 cost = 0.36746764
Epoch: 32 cost = 0.36374186
Epoch: 33 cost = 0.36067255
Epoch: 34 cost = 0.35790596
Epoch: 35 cost = 0.35517427
Epoch: 36 cost = 0.35230182
E

Epoch: 346 cost = 0.01641512
Epoch: 347 cost = 0.01639119
Epoch: 348 cost = 0.01635377
Epoch: 349 cost = 0.01637699
Epoch: 350 cost = 0.01638037
Epoch: 351 cost = 0.01627633
Epoch: 352 cost = 0.01621585
Epoch: 353 cost = 0.01618759
Epoch: 354 cost = 0.01656075
Epoch: 355 cost = 0.02129118
Epoch: 356 cost = 0.04682613
Epoch: 357 cost = 0.01657118
Epoch: 358 cost = 0.01421964
Epoch: 359 cost = 0.01408548
Epoch: 360 cost = 0.01410768
Epoch: 361 cost = 0.01406891
Epoch: 362 cost = 0.01414968
Epoch: 363 cost = 0.01418464
Epoch: 364 cost = 0.01414841
Epoch: 365 cost = 0.01419811
Epoch: 366 cost = 0.01425363
Epoch: 367 cost = 0.01422231
Epoch: 368 cost = 0.01416318
Epoch: 369 cost = 0.01414778
Epoch: 370 cost = 0.01418603
Epoch: 371 cost = 0.01412910
Epoch: 372 cost = 0.01412539
Epoch: 373 cost = 0.01434897
Epoch: 374 cost = 0.01895565
Epoch: 375 cost = 0.04666822
Epoch: 376 cost = 0.01453023
Epoch: 377 cost = 0.01220794
Epoch: 378 cost = 0.01206523
Epoch: 379 cost = 0.01208031
Epoch: 380 cos

Epoch: 661 cost = 0.00063466
Epoch: 662 cost = 0.00065492
Epoch: 663 cost = 0.00060505
Epoch: 664 cost = 0.00060041
Epoch: 665 cost = 0.00057680
Epoch: 666 cost = 0.00062030
Epoch: 667 cost = 0.00061517
Epoch: 668 cost = 0.00058280
Epoch: 669 cost = 0.00061374
Epoch: 670 cost = 0.00063678
Epoch: 671 cost = 0.00059912
Epoch: 672 cost = 0.00063468
Epoch: 673 cost = 0.00076187
Epoch: 674 cost = 0.00062985
Epoch: 675 cost = 0.00073241
Epoch: 676 cost = 0.00606269
Epoch: 677 cost = 0.01011390
Epoch: 678 cost = 0.00082664
Epoch: 679 cost = 0.00067334
Epoch: 680 cost = 0.00062106
Epoch: 681 cost = 0.00059829
Epoch: 682 cost = 0.00061660
Epoch: 683 cost = 0.00059693
Epoch: 684 cost = 0.00058123
Epoch: 685 cost = 0.00052882
Epoch: 686 cost = 0.00053672
Epoch: 687 cost = 0.00061401
Epoch: 688 cost = 0.00058647
Epoch: 689 cost = 0.00054529
Epoch: 690 cost = 0.00058305
Epoch: 691 cost = 0.00066904
Epoch: 692 cost = 0.00058107
Epoch: 693 cost = 0.00056225
Epoch: 694 cost = 0.00059024
Epoch: 695 cos

Epoch: 976 cost = 0.00045599
Epoch: 977 cost = 0.00048410
Epoch: 978 cost = 0.00039180
Epoch: 979 cost = 0.00046341
Epoch: 980 cost = 0.00050418
Epoch: 981 cost = 0.00042623
Epoch: 982 cost = 0.00037130
Epoch: 983 cost = 0.00046753
Epoch: 984 cost = 0.00046960
Epoch: 985 cost = 0.00040537
Epoch: 986 cost = 0.00047780
Epoch: 987 cost = 0.00048943
Epoch: 988 cost = 0.00042056
Epoch: 989 cost = 0.00048020
Epoch: 990 cost = 0.00043049
Epoch: 991 cost = 0.00040570
Epoch: 992 cost = 0.00047878
Epoch: 993 cost = 0.00042557
Epoch: 994 cost = 0.00040831
Epoch: 995 cost = 0.00050231
Epoch: 996 cost = 0.00048322
Epoch: 997 cost = 0.00044918
Epoch: 998 cost = 0.00047943
Epoch: 999 cost = 0.00045635
Epoch: 1000 cost = 0.00050479


In [43]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [44]:
print(y_test.shape)

(59, 5)


In [20]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.6101695


In [45]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.6440678


In [21]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data[0:204,:]
X_test = data[204:,:]
y_train = label[0:204]
y_test = label[204:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(204, 21)
(87, 21)
(204,)
(87,)


In [22]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(204, 5)
(87, 5)


In [25]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.03948120
Epoch: 2 cost = 0.84643483
Epoch: 3 cost = 0.74147155
Epoch: 4 cost = 0.68363708
Epoch: 5 cost = 0.64615818
Epoch: 6 cost = 0.61928785
Epoch: 7 cost = 0.59869854
Epoch: 8 cost = 0.58160335
Epoch: 9 cost = 0.56633327
Epoch: 10 cost = 0.55220499
Epoch: 11 cost = 0.53915956
Epoch: 12 cost = 0.52738104
Epoch: 13 cost = 0.51702138
Epoch: 14 cost = 0.50805583
Epoch: 15 cost = 0.50026103
Epoch: 16 cost = 0.49328655
Epoch: 17 cost = 0.48676276
Epoch: 18 cost = 0.48039015
Epoch: 19 cost = 0.47398649
Epoch: 20 cost = 0.46749814
Epoch: 21 cost = 0.46098924
Epoch: 22 cost = 0.45461414
Epoch: 23 cost = 0.44857043
Epoch: 24 cost = 0.44303888
Epoch: 25 cost = 0.43813300
Epoch: 26 cost = 0.43387890
Epoch: 27 cost = 0.43022839
Epoch: 28 cost = 0.42708969
Epoch: 29 cost = 0.42435815
Epoch: 30 cost = 0.42193782
Epoch: 31 cost = 0.41975063
Epoch: 32 cost = 0.41773387
Epoch: 33 cost = 0.41583161
Epoch: 34 cost = 0.41398414
Epoch: 35 cost = 0.41212213
Epoch: 36 cost = 0.41016515
E

Epoch: 349 cost = 0.00784450
Epoch: 350 cost = 0.00807665
Epoch: 351 cost = 0.00865103
Epoch: 352 cost = 0.00931773
Epoch: 353 cost = 0.00993332
Epoch: 354 cost = 0.01049505
Epoch: 355 cost = 0.01072965
Epoch: 356 cost = 0.01000503
Epoch: 357 cost = 0.00808160
Epoch: 358 cost = 0.00925830
Epoch: 359 cost = 0.01494908
Epoch: 360 cost = 0.01177692
Epoch: 361 cost = 0.00589057
Epoch: 362 cost = 0.00595463
Epoch: 363 cost = 0.00640440
Epoch: 364 cost = 0.00717448
Epoch: 365 cost = 0.00866663
Epoch: 366 cost = 0.01257108
Epoch: 367 cost = 0.02063711
Epoch: 368 cost = 0.00992787
Epoch: 369 cost = 0.00511896
Epoch: 370 cost = 0.00537959
Epoch: 371 cost = 0.00546340
Epoch: 372 cost = 0.00547258
Epoch: 373 cost = 0.00557832
Epoch: 374 cost = 0.00567359
Epoch: 375 cost = 0.00573454
Epoch: 376 cost = 0.00578731
Epoch: 377 cost = 0.00588141
Epoch: 378 cost = 0.00587474
Epoch: 379 cost = 0.00584225
Epoch: 380 cost = 0.00583936
Epoch: 381 cost = 0.00571588
Epoch: 382 cost = 0.00573643
Epoch: 383 cos

Epoch: 652 cost = 0.00033280
Epoch: 653 cost = 0.00031463
Epoch: 654 cost = 0.00037465
Epoch: 655 cost = 0.00045783
Epoch: 656 cost = 0.00047962
Epoch: 657 cost = 0.00038844
Epoch: 658 cost = 0.00041193
Epoch: 659 cost = 0.00050847
Epoch: 660 cost = 0.00270864
Epoch: 661 cost = 0.01334310
Epoch: 662 cost = 0.00065450
Epoch: 663 cost = 0.00047170
Epoch: 664 cost = 0.00045175
Epoch: 665 cost = 0.00038003
Epoch: 666 cost = 0.00032415
Epoch: 667 cost = 0.00034468
Epoch: 668 cost = 0.00035380
Epoch: 669 cost = 0.00037616
Epoch: 670 cost = 0.00037416
Epoch: 671 cost = 0.00027802
Epoch: 672 cost = 0.00038380
Epoch: 673 cost = 0.00033471
Epoch: 674 cost = 0.00033109
Epoch: 675 cost = 0.00030527
Epoch: 676 cost = 0.00027708
Epoch: 677 cost = 0.00029561
Epoch: 678 cost = 0.00036928
Epoch: 679 cost = 0.00034124
Epoch: 680 cost = 0.00027524
Epoch: 681 cost = 0.00034156
Epoch: 682 cost = 0.00033184
Epoch: 683 cost = 0.00030755
Epoch: 684 cost = 0.00035632
Epoch: 685 cost = 0.00033863
Epoch: 686 cos

Epoch: 955 cost = 0.00024834
Epoch: 956 cost = 0.00029446
Epoch: 957 cost = 0.00020256
Epoch: 958 cost = 0.00017684
Epoch: 959 cost = 0.00025024
Epoch: 960 cost = 0.00027320
Epoch: 961 cost = 0.00019036
Epoch: 962 cost = 0.00027280
Epoch: 963 cost = 0.00030957
Epoch: 964 cost = 0.00018246
Epoch: 965 cost = 0.00019629
Epoch: 966 cost = 0.00032550
Epoch: 967 cost = 0.00028340
Epoch: 968 cost = 0.00023640
Epoch: 969 cost = 0.00026600
Epoch: 970 cost = 0.00023170
Epoch: 971 cost = 0.00020615
Epoch: 972 cost = 0.00027044
Epoch: 973 cost = 0.00024850
Epoch: 974 cost = 0.00024541
Epoch: 975 cost = 0.00029768
Epoch: 976 cost = 0.00032239
Epoch: 977 cost = 0.00021697
Epoch: 978 cost = 0.00026723
Epoch: 979 cost = 0.00032195
Epoch: 980 cost = 0.00021251
Epoch: 981 cost = 0.00030588
Epoch: 982 cost = 0.00018460
Epoch: 983 cost = 0.00022741
Epoch: 984 cost = 0.00028919
Epoch: 985 cost = 0.00025899
Epoch: 986 cost = 0.00029336
Epoch: 987 cost = 0.00024670
Epoch: 988 cost = 0.00019708
Epoch: 989 cos

In [27]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.6091954
